# Quick Start Tutorial: Model Training

## Learning Objectives

In this tutorial you will learn:
1. How to design an observation set for your use case
2. How to materialize training data
3. How your ML training environment can consume training data

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalog

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

17:41:10 | INFO     | Using configuration file at: /Users/jevonyeoh/.featurebyte/config.yaml
17:41:10 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
17:41:10 | INFO     | (1/4) Starting featurebyte services
Container spark-thrift  Running
Container mongo-rs  Running
Container redis  Running
Container featurebyte-server  Running
Container featurebyte-worker  Running
Container mongo-rs  Waiting
Container redis  Waiting
Container mongo-rs  Waiting
Container mongo-rs  Healthy
Container redis  Healthy
Container mongo-rs  Healthy
17:41:12 | INFO     | (2/4) Creating local spark feature store
17:41:12 | INFO     | (3/4) Import datasets
17:41:25 | INFO     | Dataset grocery already exists, skipping import
17:41:25 | INFO     | Dataset healthcare already exists, skipping import
17:41:25 | INFO     | Dataset creditcard already exists, skipping import
17:41:25 | INFO     | (4/4) Playground environment started successfully. Ready to go

### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartModelTraining)

Cleaning up existing tutorial catalogs


17:41:26 | INFO     | Catalog activated: deep dive materializing features 20230803:1135


Cleaning catalog: deep dive materializing features 20230803:1135
  1 batch feature tables
  1 batch request tables
  1 historical feature tables
  3 observation tables
Done! |████████████████████████████████████████| 100% in 12.5s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 12.4s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 15.6s (0.06%/s)        
Done! |████████████████████████████████████████| 100% in 15.8s (0.06%/s)        
Done! |████████████████████████████████████████| 100% in 12.5s (0.08%/s)        
Done! |████████████████████████████████████████| 100% in 12.5s (0.08%/s)        


17:42:51 | INFO     | Catalog activated: quick start model training 20230803:1742


Building a quick start catalog for model training named [quick start model training 20230803:1742]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.1s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.1s
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.

### Example: Create views from tables in the Catalog

In [3]:
# create the views
grocery_customer_view = catalog.get_view("GROCERYCUSTOMER")
grocery_invoice_view = catalog.get_view("GROCERYINVOICE")
grocery_items_view = catalog.get_view("INVOICEITEMS")
grocery_product_view = catalog.get_view("GROCERYPRODUCT")

## Create an observation set for your use case

Learning Objectives

In this section you will learn:
* the purpose of observation sets
* the relationship between entities, point in time, and observation sets
* how to design an observation set suitable for training data

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

### Concept: Materialization

A feature in FeatureByte is defined by the logical plan for its computation. The act of computing the feature is known as Feature Materialization.

The materialization of features is made on demand to fulfill historical requests, whereas for prediction purposes, feature values are generated through a batch process called a "Feature Job". The Feature Job is scheduled based on the defined settings associated with each feature.

### Concept: Observation set

An observation set combines entity key values and historical points-in-time, for which you wish to materialize feature values.

The observation set can be a pandas DataFrame or an ObservationTable object representing an observation set in the feature store. An accepted serving name must be used for the column containing the entity values. The column containing points-in-time must be labelled "POINT-IN-TIME" and the point-in-time timestamps should be in UTC.

### Concept: Point in time

A point-in-time for a feature refers to a specific moment in the past with which the feature's values are associated.

It is a crucial aspect of historical feature serving, which allows machine learning models to make predictions based on historical data. By providing a point-in-time, a feature can be used to train and test models on past data, enabling them to make accurate predictions for similar situations in the future.

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

In [4]:
# get the feature list for the target
customer_target = catalog.get_target("next_customer_sales_14d")

# display details about the target
display(customer_target.info())

{
  'id': '64cb77056f181a8e9ae51c4a',
  'target_name': 'next_customer_sales_14d',
  'entities': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start model training 20230803:1742'
    }
  ],
  'window': '14d',
  'has_recipe': True,
  'created_at': '2023-08-03T09:44:37.530000',
  'updated_at': None,
  'primary_table': [
    {
      'name': 'GROCERYINVOICE',
      'status': 'PUBLIC_DRAFT',
      'catalog_name': 'quick start model training 20230803:1742'
    }
  ],
  'metadata': {
    'input_columns': {
      'Input0': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Timestamp',
        'semantic': 'event_timestamp'
      },
      'Input1': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Amount',
        'semantic': None
      }
    },
    'derived_columns': {},
    'aggregations': {
      'F0': {
        'name': 'next_customer_sales_14d',
        'column': 'Input1',
        'function': 'sum',
        'keys': [
          'GroceryCustomerGuid'
        ],
        'window': '14d',
        'category': None,
        'filter': False
      }
    },
    'post_aggregation': {
      'name': 'next_customer_sales_14d',
      'inputs': [
        'F0'
      ],
      'transforms': [
        'is_null',
        'conditional'
      ]
    }
  },
  'namespace_description': None,
  'description': None
}

In [5]:
# create a large observation table from a view

# filter the view to exclude points in time that won't have data for historical windows
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table = observation_set_view.create_observation_table(
    name="10,000 Customers immediately after each purchase from May-22 to Mar-23",
    sample_rows=10000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 25.0s (0.04%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-11 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-04-15 09:50:57,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-05-14 15:00:07,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-05-20 13:03:26,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-05-25 13:01:50,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-08-17 09:33:54,3dff3cdc-3e64-4b6f-ab62-8165afca8c55
9996,2022-09-14 08:38:30,3dff3cdc-3e64-4b6f-ab62-8165afca8c55
9997,2022-09-17 09:09:46,3dff3cdc-3e64-4b6f-ab62-8165afca8c55
9998,2022-09-19 10:20:23,3dff3cdc-3e64-4b6f-ab62-8165afca8c55


## Materialize Training Data

Learning Objectives

In this section you will learn:
* how to create a target observation table
* how to create historical training data using the target observation table

### Example: Get target values


In [6]:
# Materialize the target
training_data_target_table = customer_target.compute_target_table(
    observation_table, observation_table_name="target_observation_table"
)

display(training_data_target_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 31.7s (0.03%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d
0,2022-11-14 14:07:14,abdef773-ab72-43b6-8e77-050804c1c5fc,111.22
1,2022-11-05 19:48:29,776ed61f-ae99-40b4-989b-1195e4901090,15.42
2,2022-11-18 12:24:01,9b1b8037-8506-4a54-981a-3b7e694a489f,81.43
3,2022-07-25 11:16:04,94127b9f-1366-4bbe-afea-7cd77225da52,53.55
4,2022-11-10 13:34:16,b6cb759f-20d0-4a1f-902c-f62881bda7e6,72.80
...,...,...,...
9995,2022-06-08 09:32:33,c5aba9b4-c5d2-4639-abb7-484de6ab8d85,0.00
9996,2022-10-23 09:37:54,df23dfe3-fede-46aa-b283-422458c11767,37.35
9997,2022-09-19 10:22:06,4d59c319-966b-4f24-a5f0-a2245532064a,21.13
9998,2022-06-20 06:07:19,f56f0029-0cf3-4c28-bbec-2a8292722a32,4.07


### Example: Get historical values with target

In [7]:
# list the feature lists
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64cb76fe6f181a8e9ae51c46,Features,8,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-03T09:44:31.098000


In [8]:
# get the feature list
feature_list = catalog.get_feature_list("Features")

Loading Feature(s) |████████████████████████████████████████| 8/8 [100%] in 0.2s


In [9]:
# Compute the historical feature table by passing in the observation table that contains the target values
training_table_features = feature_list.compute_historical_feature_table(
    training_data_target_table,
    historical_feature_table_name="customer training table - invoices Apr-22 to Mar-23 - features only",
)

# display the training data
training_data = training_table_features.to_pandas()
display(training_data)

Done! |████████████████████████████████████████| 100% in 2:39.4 (0.01%/s)       


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d
0,2022-04-01 09:57:05,ed56f1f6-310d-4b7c-9f5b-554103282f15,94.06,3,15.970000,48.354199,-1.871965,1.000000,0.871330,35.84,35.840000
1,2022-04-01 10:41:29,8759ff7c-4cad-44e7-82dd-f89c925699be,15.66,14,15.516444,43.404298,3.330159,0.931614,0.578746,54.94,10.988000
2,2022-04-01 11:29:55,4b348211-553b-4831-8463-8a1e936f67d4,12.85,183,18.001381,48.740582,2.237559,0.842424,0.462505,47.62,7.936667
3,2022-04-01 11:38:19,09fbee0c-521e-40ee-a2ff-8ed4187dcbc4,88.02,8,17.258158,48.789776,5.855939,0.877520,0.802827,268.19,26.819000
4,2022-04-01 12:37:29,9e27fb57-73e7-4571-85d2-beb8ed343784,41.02,56,19.830249,43.452577,5.848259,0.118678,0.405181,66.16,16.540000
...,...,...,...,...,...,...,...,...,...,...,...
9995,2022-12-31 11:45:37,b2fb8dec-4307-4008-9aa4-bd443bd48bba,62.03,8,18.651111,48.815086,4.386779,0.702402,0.776213,242.80,60.700000
9996,2022-12-31 12:31:44,5060f005-4397-458b-b8ed-2cfd5ad437fd,7.13,13,17.396027,47.182230,4.394402,0.882227,0.675283,120.52,10.956364
9997,2022-12-31 16:06:23,c22fa3eb-55a5-4a4f-9301-38f6b6f0567e,111.11,13,17.183514,47.182230,4.394402,0.203653,0.730632,205.54,34.256667
9998,2022-12-31 17:59:15,a807b07b-09e9-4cf3-8ad6-f1fab36db9bd,67.05,4,19.663929,49.185354,-0.608146,0.264002,0.960702,390.92,78.184000


## Consuming training data

Learning Objectives

In this section you will learn:
* how to save a training file
* how to use a pandas data frame

### Example: Save the training data to a file

In [10]:
# save training data as a csv file
training_data.to_csv("training_data.csv", index=False)

In [11]:
# save the training file as a parquet file
training_data.to_parquet("training_data.parquet")

### Example: Training a scikit learn model

Note that you will need to install scikit learn https://scikit-learn.org/stable/install.html

In [12]:
# EDA on the training data
training_data.describe()

,next_customer_sales_14d,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9610.000000
mean,65.826766,80.067000,18.208290,45.291951,3.425420,0.760045,0.593087,136.153021,18.222606
std,69.302225,75.753859,3.681104,9.926533,8.917575,0.298445,0.221189,123.895564,14.764153
min,0.000000,1.000000,3.887857,-12.713308,-50.017299,0.000000,0.000000,0.000000,0.000000
25%,14.690000,14.000000,16.542690,44.663768,2.237559,0.729800,0.483684,43.620000,8.460167
50%,44.500000,33.000000,17.793871,48.211446,2.241215,0.884652,0.632729,98.515000,14.797143
75%,93.542500,180.000000,20.153644,48.739799,5.054081,0.945036,0.748809,194.447500,23.269375
max,487.200000,183.000000,47.358750,50.669452,45.189819,1.000000,1.000000,837.360000,332.300000


In [13]:
# do any columns in the training data contain missing values?
training_data.isna().any()

POINT_IN_TIME                        False
GROCERYCUSTOMERGUID                  False
next_customer_sales_14d              False
StatePopulation                      False
StateAvgInvoiceAmount_28d            False
StateMeanLatitude                    False
StateMeanLongitude                   False
CustomerInventoryStability_14d28d    False
CustomerStateSimilarity_28d          False
CustomerSpend_28d                    False
CustomerAvgInvoiceAmount_28d          True
dtype: bool

In [14]:
! pip install scikit-learn

In [15]:
# use sklearn to train a random forest regression model on the training data
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    training_data.drop(columns=["GROCERYCUSTOMERGUID", "POINT_IN_TIME"]),
    training_data["next_customer_sales_14d"],
    test_size=0.2,
    random_state=42,
)

# train the model
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train)

# get predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

# save the model
import joblib

joblib.dump(model, "model.pkl")

Mean squared error:  4.012782666469337


['model.pkl']

## Next Steps

Now that you've completed the quick-start feature engineering tutorial, you can put your knowledge into practice or learn more:<br>
1. Learn more about materializing features via the "Deep Dive Materializing Features" tutorial
2. Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" workspaces
3. Learn more about feature governance via the "Quick Start Feature Governance" tutorial